In [ ]:
import pandas as pd
import deepxde as dde
import numpy as np
import re
import matplotlib.pyplot as plt
from deepxde.backend import torch

In [ ]:
Kt = dde.Variable(1.0)
Rs = dde.Variable(1.0)
Rsh = dde.Variable(1.0)
Iso = dde.Variable(1.0)

In [ ]:
Var_list = [Kt, Rs, Rsh, Iso]

In [ ]:
# '''
# 24th July, 2023 created by Likun Chen, Wuhan University
# This is for testing the PINN parameter estimation, object: syn-machine
# EXT Parameters:
#     Vi , initial terminal voltage : 1.0 pu
#     Ta , voltage regulator time constant : 0.4 sec
#     Kf , rate feedback gain : 0.03
# SYN Parameters:
#     H , inertia constant : 1.7 MWs/MVA
#     D , synchronous mechanical damping : 0.12 pu/pu
#     Xa , stator leakage reactance : 0.130 pu
#     Xd , d-axis unsaturated reactance : 1.79 pu
#     Xd' , d-axis unsaturated transient reactance : 0.169 pu
#     Xd'' , d-axis unsaturated Sub-Trans reactance : 0.135 pu
#     Xq , q-axis unsaturated reactance : 1.71 pu
#     Xq' , q-axis unsaturated transient reactance : 0.228 pu
#     Xq'' , q-axis unsaturated Sub-Trans reactance : 0.2 pu
# For 2-order equation, the state variables are rev (w) and phase angle (delta), 
# parameters waiting estimated are H , D , Eq' = Ed' = const
#     H * d2_delta/dt2 (This is dw/dt) + D * d_delta/dt + P_ex - P_mach = 0
#     P_ex is external power balance and P_mach is the mechanical power
#     time， P_ex, delta, P_mach, dw
# '''

In [ ]:
input_data = pd.read_csv('data/1011PVtest.csv')
input_data

In [ ]:
step_time = input_data.Time[1] - input_data.Time[0]
st = 0.4 # start time (10% proportion)
et = 0.6 # end time
input_data.Time -= step_time * int(input_data.shape[0] * st)
input_data = input_data[int(input_data.shape[0]*st): int(input_data.shape[0]*et)]

In [ ]:
input_data.columns = ['Time', 'V', 'I', 'Is']
input_data.drop(columns='Is', inplace=True)
input_data.set_index('Time', inplace=True)
input_data

In [ ]:
weight_list = (1 / input_data.describe().loc['mean']).tolist()
weight_list

In [ ]:
input_data.plot()

In [ ]:
x = input_data.index.to_numpy()
x

In [ ]:
geom = dde.geometry.TimeDomain(0, x[-1])

In [ ]:
# def boundary(_, on_initial):
#     return on_initial

In [ ]:
# x0 = input_data.iloc[0].tolist()

In [ ]:
# # Initial conditions
# ic1 = dde.icbc.IC(geom, lambda X: x0[0], boundary, component=0)
# ic2 = dde.icbc.IC(geom, lambda X: x0[1], boundary, component=1)
# ic3 = dde.icbc.IC(geom, lambda X: x0[2], boundary, component=2)

In [ ]:
y = input_data.to_numpy()
y.shape

In [ ]:
observe_t = x.reshape(-1, 1)

In [ ]:
# Get the training data
observe_y0 = dde.icbc.PointSetBC(observe_t, y[:, 0:1], component=0)
observe_y1 = dde.icbc.PointSetBC(observe_t, y[:, 1:2], component=1)
# observe_y2 = dde.icbc.PointSetBC(observe_t, y[:, 2:3], component=2)

In [ ]:
# Kt = dde.Variable(0.0)
# Rs = dde.Variable(0.0)
# Rsh = dde.Variable(0.0)
def Microgrid_system(x, y):
    """Modified Lorenz system (with exogenous input).
    H * d2_delta/dt2 (This is dw/dt) + D * d_delta/dt + P_ex - P_mach = 0
    """
    V, I = y[:, 0:1], y[:, 1:2]
    
    
    dV_dt = dde.grad.jacobian(y, x, i=0)
    dI_dt = dde.grad.jacobian(y, x, i=1)

    left = 1 + Iso*Rs/Kt*torch.exp((V+I*Rs)/Kt)  + Rs/Rsh
    right = - dV_dt / dI_dt * (Iso/Kt*torch.exp((Rs*I+V)/Kt) + 1/Rsh)
    
    return [
        left - right
    ]


In [ ]:
data = dde.data.PDE(
    geom,
    Microgrid_system,
    [observe_y0, observe_y1],
    anchors=observe_t,
)

In [ ]:
net = dde.nn.FNN([2] + [128] * 3 + [2], "swish", "Glorot normal")

In [ ]:
data_y = y
data_t = x

In [ ]:
y.shape

In [ ]:
def feature_transform(t):
    t = 0.01 * t
    return torch.concat(
        (torch.sin(t), torch.sin(2 * t)),
        axis=1,
    )

net.apply_feature_transform(feature_transform)

def output_transform(t, y):
    idx = data_y.shape[0]-1
    k = (data_y[idx] - data_y[0]) / (data_t[idx] - data_t[0])
    b = (data_t[idx] * data_y[0] - data_t[0] * data_y[idx]) / (
        data_t[idx] - data_t[0]
    )
    linear = torch.as_tensor(k) * t + torch.as_tensor(b)
    factor = torch.tanh(t) * torch.tanh(idx - t)
    return linear + factor * torch.Tensor([1., 1.]) * y

net.apply_output_transform(output_transform)

In [ ]:

model = dde.Model(data, net)
model.compile("adam", lr=0.001, loss_weights=weight_list + [1e-2], external_trainable_variables=Var_list)



In [ ]:
fnamevar = "variables_testclk1.dat"
variable = dde.callbacks.VariableValue(Var_list, period=1000, filename=fnamevar)
model.train(iterations=100000, callbacks=[variable])

In [ ]:
lines = open(fnamevar, "r").readlines()
# read output data in fnamevar (this line is a long story...)
Chat = np.array(
    [
        np.fromstring(
            min(re.findall(re.escape("[") + "(.*?)" + re.escape("]"), line), key=len),
            sep=",",
        )
        for line in lines
    ]
)

In [ ]:
Chat[-10:, :]

In [ ]:
l, c = Chat.shape
plt.plot(range(l), Chat[:, 0], "r-")
plt.plot(range(l), Chat[:, 1], "b-")
plt.plot(range(l), Chat[:, 2], "y-")
plt.plot(range(l), Chat[:, 3], "g-")
# plt.plot(range(l), np.ones(Chat[:, 0].shape) * 1.5, "r--")
# plt.plot(range(l), np.ones(Chat[:, 1].shape) * 0.15, "b--")
plt.legend(["H","D","True H","True D"], loc="right")
plt.xlabel("Epoch")

In [ ]:
Chat[-10:, 0:2]

In [ ]:
yhat = model.predict(observe_t)
plt.figure()
plt.plot(observe_t, y, "-", observe_t, yhat, "--")
plt.xlabel("Time")
# plt.legend(["x", "y", "z", "xh", "yh", "zh"])
# plt.title("Training data")
plt.show()

In [ ]:
input_data = pd.read_csv('data/1011PVtest.csv')
input_data.columns = ['Time', 'V', 'I', 'Iso']
input_data

In [ ]:
Kt = dde.Variable(0.0)
Rs = dde.Variable(0.0)
Rsh = dde.Variable(0.0)

In [ ]:
Iso = input_data.Iso.to_numpy()[0]
Iso

In [ ]:
1 +  Iso * 